# Data Clean Up 

In [27]:
# Import modules
import os
import pandas as pd

## Make library data set

Thoughts
- Summary statistics of library stuff
- Totals per month of cases, tests, and deaths
- Zip code map of stats 

In [28]:
# Set parameters
covid_path = "../Resources/Weekly_COVID-19_Cases__Tests__and_Deaths_by_ZIP_Code_pulled_272022.csv"
library_circulation_2019_path = "../Resources/Libraries_-_2019_Circulation_by_Location_pulled_272022.csv"
library_visitors_2019_path = "../Resources/Libraries_-_2019_Visitors_by_Location_pulled_272022.csv"
library_circulation_2020_path = "../Resources/Libraries_-_2020_Circulation_by_Location_pulled_272022.csv"
library_visitors_2020_path = "../Resources/Libraries_-_2020_Visitors_by_Location_pulled_272022.csv"
library_circulation_2021_path = "../Resources/Libraries_-_2021_Circulation_by_Location_pulled_272022.csv"
library_visitors_2021_path = "../Resources/Libraries_-_2021_Visitors_by_Location_pulled_272022.csv"

In [29]:
# Load datasets
library_circulation_2019_df = pd.read_csv(library_circulation_2019_path, dtype={'ZIP': object})
library_visitors_2019_df = pd.read_csv(library_visitors_2019_path, dtype={'ZIP': object})
library_circulation_2020_df = pd.read_csv(library_circulation_2020_path, dtype={'ZIP': object})
library_visitors_2020_df = pd.read_csv(library_visitors_2020_path, dtype={'ZIP': object})
library_circulation_2021_df = pd.read_csv(library_circulation_2021_path, dtype={'ZIP': object})
library_visitors_2021_df = pd.read_csv(library_visitors_2021_path, dtype={'ZIP': object})

In [30]:
# For library circulation we have data without zip code (online renewal, media download, etc.)
# We want to drop this data since it has no zip code associated with it
library_circulation_2019_df = library_circulation_2019_df.dropna(how='any')
library_circulation_2020_df = library_circulation_2020_df.dropna(how='any')
library_circulation_2021_df = library_circulation_2021_df.dropna(how='any')

In [31]:
# Change LOCATION to BRANCH for 2019 data to match 2020 and 2021 format
library_circulation_2019_df = library_circulation_2019_df.rename(columns={"LOCATION": "BRANCH"})
library_visitors_2019_df = library_circulation_2019_df.rename(columns={"LOCATION": "BRANCH"})

# Change Location to LOCATION
library_circulation_2021_df = library_circulation_2021_df.rename(columns={"Location": "LOCATION"})

In [32]:
# Add type column
library_circulation_2019_df["TYPE"] = ["Circulation"] * len(library_circulation_2019_df)
library_visitors_2019_df["TYPE"] = ["Visitors"] * len(library_visitors_2019_df)
library_circulation_2020_df["TYPE"] = ["Circulation"] * len(library_circulation_2020_df)
library_visitors_2020_df["TYPE"] = ["Visitors"] * len(library_visitors_2020_df)
library_circulation_2021_df["TYPE"] = ["Circulation"] * len(library_circulation_2021_df)
library_visitors_2021_df["TYPE"] = ["Visitors"] * len(library_visitors_2021_df)

# Add year column
library_circulation_2019_df["YEAR"] = [2019] * len(library_circulation_2019_df)
library_visitors_2019_df["YEAR"] = [2019] * len(library_visitors_2019_df)
library_circulation_2020_df["YEAR"] = [2020] * len(library_circulation_2020_df)
library_visitors_2020_df["YEAR"] = [2020] * len(library_visitors_2020_df)
library_circulation_2021_df["YEAR"] = [2021] * len(library_circulation_2021_df)
library_visitors_2021_df["YEAR"] = [2021] * len(library_visitors_2021_df)

In [33]:
# Merge circulation and visitors together
merged_2019 = pd.merge(library_circulation_2019_df, library_visitors_2019_df, how = "outer")
merged_2020 = pd.merge(library_circulation_2020_df, library_visitors_2020_df, how = "outer")
merged_2021 = pd.merge(library_circulation_2021_df, library_visitors_2021_df, how = "outer")

# Merge all together
merged = pd.merge(merged_2019, merged_2020, how = "outer")
merged = pd.merge(merged, merged_2021, how = "outer")

# Remove erroneous columns
merged = merged.drop(['LOCATION'], axis = 1)

# Remove index column
merged = merged.set_index('BRANCH')
merged.head()

,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR
BRANCH,,,,,,,,,,,,,,,,,,
Albany Park,3401 W. Foster Ave.,Chicago,60625,8214,7614,8460,7414,7044,7970,9664,9069,7053,8012,6571,5924,93009,Circulation,2019
Altgeld,13281 S. Corliss Ave.,Chicago,60827,378,326,332,270,320,325,354,229,234,377,369,500,4014,Circulation,2019
Archer Heights,5055 S. Archer Ave.,Chicago,60632,5365,5019,5417,5349,4833,4627,5356,4793,4537,5263,4051,3415,58025,Circulation,2019
Austin,5615 W. Race Ave.,Chicago,60644,215,590,706,736,806,983,1134,1236,1193,1361,1286,1082,11328,Circulation,2019
Austin-Irving,6100 W. Irving Park Rd.,Chicago,60634,9678,9764,10845,10541,9763,11442,11634,11063,9379,9932,8567,6492,119100,Circulation,2019


## Make COVID data set

In [34]:
# Load data
covid_df = pd.read_csv(covid_path)

# Align zip code column name
covid_df = covid_df.rename(columns={"ZIP Code": "ZIP"})

# Designate month and year using Week Start 
covid_df['YEAR'] = pd.to_datetime(covid_df['Week Start']).dt.year
covid_df['Month'] = pd.to_datetime(covid_df['Week Start']).dt.month
covid_df.head()

,ZIP,Week Number,Week Start,Week End,Cases - Weekly,Cases - Cumulative,Case Rate - Weekly,Case Rate - Cumulative,Tests - Weekly,Tests - Cumulative,...,Percent Tested Positive - Cumulative,Deaths - Weekly,Deaths - Cumulative,Death Rate - Weekly,Death Rate - Cumulative,Population,Row ID,ZIP Code Location,YEAR,Month
0,60612,15,4/5/2020,4/11/2020,69.0,202.0,201.0,588.7,284.0,803,...,0.3,3,7,8.7,20.4,34311,60612-2020-15,POINT (-87.687011 41.88004),2020,4
1,60612,16,4/12/2020,4/18/2020,66.0,268.0,192.0,781.1,286.0,1089,...,0.3,3,10,8.7,29.1,34311,60612-2020-16,POINT (-87.687011 41.88004),2020,4
2,60604,4,1/23/2022,1/29/2022,8.0,322.0,1023.0,41176.5,246.0,11461,...,0.0,0,0,0.0,0.0,782,60604-2022-4,POINT (-87.629029 41.878153),2022,1
3,60615,24,6/7/2020,6/13/2020,12.0,377.0,29.0,907.1,538.0,3986,...,0.1,0,15,0.0,36.1,41563,60615-2020-24,POINT (-87.602725 41.801993),2020,6
4,60615,25,6/14/2020,6/20/2020,13.0,390.0,31.0,938.3,551.0,4537,...,0.1,1,16,2.4,38.5,41563,60615-2020-25,POINT (-87.602725 41.801993),2020,6


In [35]:
# Group and sum cases-weekly for a given year, month, and zip code
covid_grouped = covid_df.groupby(['YEAR', 'Month', 'ZIP'])
cases_weekly = pd.DataFrame(covid_grouped['Cases - Weekly'].sum())
cases_weekly = cases_weekly.reset_index()
cases_weekly_formatted = cases_weekly[ cases_weekly['Month'] == 1]
cases_weekly_formatted = cases_weekly_formatted.drop(['Month'], axis = 1)
cases_weekly_formatted = cases_weekly_formatted.rename(columns={"Cases - Weekly": "JANUARY"})

month_conversion = {2: "FEBRUARY",
                    3: "MARCH",
                    4: "APRIL",
                    5: "MAY",
                    6: "JUNE",
                    7: "JULY",
                    8: "AUGUST",
                    9: "SEPTEMBER",
                    10: "OCTOBER",
                    11: "NOVEMBER",
                    12: "DECEMBER"}

for month in range(2,13):
    cases_weekly_subset = cases_weekly[ cases_weekly['Month'] == month]
    cases_weekly_subset = cases_weekly_subset.drop(['Month'], axis = 1)
    cases_weekly_formatted = pd.merge(cases_weekly_formatted, cases_weekly_subset, how = "outer", on = ["YEAR", "ZIP"])
    cases_weekly_formatted = cases_weekly_formatted.rename(columns={"Cases - Weekly": month_conversion[month]})
    
cases_weekly_formatted['TYPE'] = ['Cases'] * len(cases_weekly_formatted)
cases_weekly_formatted['YTD'] = cases_weekly_formatted.iloc[:,2:14].sum(axis = 1)
cases_weekly_formatted.tail()

,YEAR,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,TYPE,YTD
175,2020,60661,NaN,NaN,12.0,13.0,15.0,16.0,43.0,26.0,38.0,111.0,217.0,73.0,Cases,564.0
176,2020,60666,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cases,0.0
177,2020,60707,NaN,NaN,28.0,171.0,170.0,37.0,49.0,92.0,66.0,247.0,649.0,283.0,Cases,1792.0
178,2020,60827,NaN,NaN,12.0,30.0,37.0,5.0,13.0,7.0,7.0,29.0,62.0,45.0,Cases,247.0
179,2020,Unknown,NaN,NaN,5.0,41.0,44.0,13.0,12.0,18.0,17.0,66.0,223.0,153.0,Cases,592.0


In [36]:
# Group and sum deaths-weekly for a given year, month, and zip code
covid_grouped = covid_df.groupby(['YEAR', 'Month', 'ZIP'])
deaths_weekly = pd.DataFrame(covid_grouped['Deaths - Weekly'].sum())
deaths_weekly = deaths_weekly.reset_index()
deaths_weekly_formatted = deaths_weekly[ deaths_weekly['Month'] == 1]
deaths_weekly_formatted = deaths_weekly_formatted.drop(['Month'], axis = 1)
deaths_weekly_formatted = deaths_weekly_formatted.rename(columns={"Deaths - Weekly": "JANUARY"})

for month in range(2,13):
    deaths_weekly_subset = deaths_weekly[ deaths_weekly['Month'] == month]
    deaths_weekly_subset = deaths_weekly_subset.drop(['Month'], axis = 1)
    deaths_weekly_formatted = pd.merge(deaths_weekly_formatted, deaths_weekly_subset, how = "outer", on = ["YEAR", "ZIP"])
    deaths_weekly_formatted = deaths_weekly_formatted.rename(columns={"Deaths - Weekly": month_conversion[month]})
    
deaths_weekly_formatted['TYPE'] = ['Deaths'] * len(deaths_weekly_formatted)
deaths_weekly_formatted['YTD'] = deaths_weekly_formatted.iloc[:,2:14].sum(axis = 1)
deaths_weekly_formatted.tail()

,YEAR,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,TYPE,YTD
175,2020,60661,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Deaths,1.0
176,2020,60666,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Deaths,0.0
177,2020,60707,NaN,NaN,1.0,3.0,8.0,0.0,0.0,0.0,1.0,1.0,3.0,17.0,Deaths,34.0
178,2020,60827,NaN,NaN,1.0,2.0,4.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,Deaths,11.0
179,2020,Unknown,NaN,NaN,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Deaths,6.0


In [37]:
# Group and sum deaths-weekly for a given year, month, and zip code
covid_grouped = covid_df.groupby(['YEAR', 'Month', 'ZIP'])
tests_weekly = pd.DataFrame(covid_grouped['Tests - Weekly'].sum())
tests_weekly = tests_weekly.reset_index()
tests_weekly_formatted = tests_weekly[ tests_weekly['Month'] == 1]
tests_weekly_formatted = tests_weekly_formatted.drop(['Month'], axis = 1)
tests_weekly_formatted = tests_weekly_formatted.rename(columns={"Tests - Weekly": "JANUARY"})

for month in range(2,13):
    tests_weekly_subset = tests_weekly[ tests_weekly['Month'] == month]
    tests_weekly_subset = tests_weekly_subset.drop(['Month'], axis = 1)
    tests_weekly_formatted = pd.merge(tests_weekly_formatted, tests_weekly_subset, how = "outer", on = ["YEAR", "ZIP"])
    tests_weekly_formatted = tests_weekly_formatted.rename(columns={"Tests - Weekly": month_conversion[month]})
    
tests_weekly_formatted['TYPE'] = ['Tests'] * len(tests_weekly_formatted)
tests_weekly_formatted['YTD'] = tests_weekly_formatted.iloc[:,2:14].sum(axis = 1)
tests_weekly_formatted.tail()

,YEAR,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,TYPE,YTD
175,2020,60661,NaN,NaN,67.0,111.0,303.0,473.0,844.0,1119.0,1030.0,1595.0,2620.0,1938.0,Tests,10100.0
176,2020,60666,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tests,0.0
177,2020,60707,NaN,NaN,117.0,601.0,1278.0,901.0,1287.0,205.0,0.0,0.0,0.0,0.0,Tests,4389.0
178,2020,60827,NaN,NaN,34.0,115.0,220.0,163.0,233.0,42.0,0.0,0.0,0.0,0.0,Tests,807.0
179,2020,Unknown,NaN,NaN,1984.0,4977.0,11647.0,10427.0,11721.0,9411.0,6414.0,7048.0,10896.0,8235.0,Tests,82760.0


In [38]:
# Merge library and covid data
merged = pd.merge(merged, cases_weekly_formatted, how = "outer")
merged = pd.merge(merged, deaths_weekly_formatted, how = "outer")
merged = pd.merge(merged, tests_weekly_formatted, how = "outer")
merged.tail()

,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR
1020,NaN,NaN,60661,NaN,NaN,67.0,111.0,303.0,473.0,844.0,1119.0,1030.0,1595.0,2620.0,1938.0,10100.0,Tests,2020
1021,NaN,NaN,60666,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tests,2020
1022,NaN,NaN,60707,NaN,NaN,117.0,601.0,1278.0,901.0,1287.0,205.0,0.0,0.0,0.0,0.0,4389.0,Tests,2020
1023,NaN,NaN,60827,NaN,NaN,34.0,115.0,220.0,163.0,233.0,42.0,0.0,0.0,0.0,0.0,807.0,Tests,2020
1024,NaN,NaN,Unknown,NaN,NaN,1984.0,4977.0,11647.0,10427.0,11721.0,9411.0,6414.0,7048.0,10896.0,8235.0,82760.0,Tests,2020


In [39]:
# Merge ZIP code coordinates
merged = pd.merge(merged, covid_df[["ZIP", "ZIP Code Location"]].drop_duplicates(), how = "left")
lat = merged["ZIP Code Location"].str.split(expand = True)[1].str.replace("(", "", regex = False)
lng = merged["ZIP Code Location"].str.split(expand = True)[2].str.replace(")", "", regex = False)
merged["LAT"] = lat
merged["LNG"] = lng
merged = merged.drop(['ZIP Code Location'], axis = 1)
merged.head()

,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR,LAT,LNG
0,3401 W. Foster Ave.,Chicago,60625,8214.0,7614.0,8460.0,7414.0,7044.0,7970.0,9664.0,9069.0,7053.0,8012.0,6571.0,5924.0,93009.0,Circulation,2019,-87.701816,41.971155
1,13281 S. Corliss Ave.,Chicago,60827,378.0,326.0,332.0,270.0,320.0,325.0,354.0,229.0,234.0,377.0,369.0,500.0,4014.0,Circulation,2019,-87.633087,41.650765
2,5055 S. Archer Ave.,Chicago,60632,5365.0,5019.0,5417.0,5349.0,4833.0,4627.0,5356.0,4793.0,4537.0,5263.0,4051.0,3415.0,58025.0,Circulation,2019,-87.711251,41.810038
3,5615 W. Race Ave.,Chicago,60644,215.0,590.0,706.0,736.0,806.0,983.0,1134.0,1236.0,1193.0,1361.0,1286.0,1082.0,11328.0,Circulation,2019,-87.756863,41.881113
4,6100 W. Irving Park Rd.,Chicago,60634,9678.0,9764.0,10845.0,10541.0,9763.0,11442.0,11634.0,11063.0,9379.0,9932.0,8567.0,6492.0,119100.0,Circulation,2019,-87.797373,41.944967


In [40]:
# Export data
merged.to_csv("../Data/merged_library_covid_outer.csv")

## Make sidewalk cafe permit data

Tasks:

- Summary statistics: Jeff
- temporal: Line graph tracking cases and/or deaths OVER TIME + number of library rentals and visitors: Brian K.
- Correlation between cases and/or deaths + library rentals and/or visitors: Natalie + other stats
- spatial: map plot neighborhood/zip code delineations and/or heatmap for three (maybe also summer) stages of pandemic, even more monthly breakdowns: Brian Z.
- Repeat above for sidewalk cafe permits
- Investigate sidewalk cafe permits + weird circulation data (automated collection): Josh

In [41]:
cafe_path = "../Resources/Sidewalk_Cafe_Permits_pulled_272022.csv"
cafe_df = pd.read_csv(cafe_path)
cafe_df.head()

,PERMIT NUMBER,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ISSUED DATE,EXPIRATION DATE,PAYMENT DATE,ADDRESS,ADDRESS NUMBER START,...,STREET,STREET TYPE,CITY,STATE,ZIP CODE,WARD,POLICE DISTRICT,LATITUDE,LONGITUDE,LOCATION
0,1000571,201243,1,THE 5 BOROUGHSDELI LLC,THE 5 BOROUGHS DELICATESSEN,4/3/2001,11/1/2001,10/18/2001,738 N WELLS ST,738,...,WELLS,ST,CHICAGO,IL,60654.0,42,18.0,41.895920,-87.634330,"(41.89592016641943, -87.63433035556712)"
1,1001307,202156,1,Rick's Cafe-Casablanca,Rick's Cafe-Casablanca,6/18/2001,11/1/2001,6/18/2001,3915 N SHERIDAN RD,3915,...,SHERIDAN,RD,CHICAGO,IL,60613.0,46,19.0,41.953158,-87.654359,"(41.953157536576526, -87.6543589916925)"
2,1002652,203574,1,cafe mauro inc,CAFE MAURO,5/7/2001,11/1/2001,5/7/2001,3625 W 26TH ST,3625,...,26TH,ST,CHICAGO,IL,60623.0,22,10.0,41.844259,-87.715946,"(41.84425865447129, -87.71594584494551)"
3,1002993,51,1,152 E. ONTARIO CORP.,O'NEIL'S BAR & GRILL,4/4/2001,11/1/2001,4/4/2001,152 E ONTARIO ST,152,...,ONTARIO,ST,CHICAGO,IL,60611.0,42,18.0,41.893460,-87.623374,"(41.89345958880933, -87.62337423955238)"
4,1003612,204664,1,"A PITZALLA BLUES, INC.",THE REVEL ROOM,8/1/2001,11/1/2001,8/1/2001,1566 N MILWAUKEE AVE,1566,...,MILWAUKEE,AVE,CHICAGO,IL,60622.0,1,14.0,41.909880,-87.676914,"(41.90987981937775, -87.67691379793119)"
